**Set environment**

In [1]:
source ../config/config_duke.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY:     /gpfs/fs1/data/reddylab/Kuei
PATH OF SOURCE:     /gpfs/fs1/data/reddylab/Kuei/source
PATH OF EXECUTABLE: /gpfs/fs1/data/reddylab/Kuei/exe
PATH OF ANNOTATION: /gpfs/fs1/data/reddylab/Kuei/annotation
PATH OF PROJECT:    /gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_ENCODE_FCC/notebooks
PATH OF RESULTS:    /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc



## Sum up the score column in the bed file

**TEST**

In [2]:
FPATH=${FP_WGS_WSTARR[0]}
FNAME=$(basename ${FPATH})
echo ${FPATH}
echo ++++++++++++++++++++++++
echo ${FNAME}
echo ++++++++++++++++++++++++
zcat ${FPATH} | head -n 10

/data/reddylab/Alex/encode4_duke/data/starr_seq/fragments/A001-input-K562-rep1.masked.dedup.fragments.counts.txt.gz
++++++++++++++++++++++++
A001-input-K562-rep1.masked.dedup.fragments.counts.txt.gz
++++++++++++++++++++++++
chr1	10335	10549	chr1_10335_10549	1	.
chr1	16983	17500	chr1_16983_17500	1	.
chr1	16985	17499	chr1_16985_17499	1	.
chr1	17004	17500	chr1_17004_17500	1	.
chr1	17483	17898	chr1_17483_17898	1	.
chr1	17487	17941	chr1_17487_17941	1	.
chr1	33606	33997	chr1_33606_33997	1	.
chr1	33617	33997	chr1_33617_33997	1	.
chr1	38785	39226	chr1_38785_39226	1	.
chr1	46328	46678	chr1_46328_46678	1	.

gzip: stdout: Broken pipe


In [3]:
zcat ${FPATH} | head -n 10 | awk -v OFS=, -v FNAME=${FNAME} '{sum += $5} END {print FNAME, sum}'

A001-input-K562-rep1.masked.dedup.fragments.counts.txt.gz,10

gzip: stdout: Broken pipe


In [4]:
echo "${#FP_WGS_WSTARR[@]}"

7


**RUN**

In [5]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 20G \
    --output ${FD_LOG}/coverage_library_size_WSTARR.txt \
    --array 0 \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh

PREFIX=A001_K562_WSTARRseq
FD_OUT=${FD_RES}/${PREFIX}/coverage/library_size
FN_OUT=library_size.csv
FP_OUT=${FD_OUT}/${FN_OUT}

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### init
mkdir -p ${FD_OUT}
echo -n > ${FP_OUT}

### get library size
for FP_INP in ${FP_WGS_WSTARR[@]}; do
    
    ### show I/O file
    echo ++++++++++++++++++++++++++++++++++++
    echo "Input file path: ${FP_INP}"
    echo "Show the first few lines"
    zcat ${FP_INP} | head -n 3
    echo
    
    ### sum up counts of each fragment
    FN_INP=$(basename ${FP_INP})
    zcat ${FP_INP} |\
        awk -v OFS=, -v FNAME=${FN_INP} '{sum += $5} END {print FNAME, sum}' \
        >> ${FP_OUT}
done

### show I/O file
echo "Output file path: ${FP_OUT}"
echo "Show the first few lines"
cat ${FP_OUT}
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 29157595


**CHECK**

In [7]:
cat ${FD_LOG}/coverage_library_size_WSTARR.txt

Hostname:           x1-01-3.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         10-16-22+12:44:20

++++++++++++++++++++++++++++++++++++
Input file path: /data/reddylab/Alex/encode4_duke/data/starr_seq/fragments/A001-input-K562-rep1.masked.dedup.fragments.counts.txt.gz
Show the first few lines
chr1	10335	10549	chr1_10335_10549	1	.
chr1	16983	17500	chr1_16983_17500	1	.
chr1	16985	17499	chr1_16985_17499	1	.

++++++++++++++++++++++++++++++++++++
Input file path: /data/reddylab/Alex/encode4_duke/data/starr_seq/fragments/A001-input-K562-rep2.masked.dedup.fragments.counts.txt.gz
Show the first few lines
chr1	10244	10370	chr1_10244_10370	1	.
chr1	14494	14961	chr1_14494_14961	1	.
chr1	15593	16258	chr1_15593_16258	1	.

++++++++++++++++++++++++++++++++++++
Input file path: /data/reddylab/Alex/encode4_duke/data/starr_seq/fragments/A001-input-K562-rep3.masked.dedup.fragments.counts.txt.gz
Show the first few lines
chr1	10247	10299	chr1_10247_10299	1	.
chr1	14684	15184	chr1_14684_15184	1	.
chr1